# 1. Building the DB with postgreSQL

### Library imports

In [15]:
from modules import download_most_recent_pgdump
from modules import connect_execute_disconnect_from_psql
from modules import retrieve_config_parameters
from modules import restore_db
from modules import save_db

import time

### 1. Downloading the Derpibooru DB

The database we will be using is the DerpiBooru database, which is made available as a nightly postgreSQL dump (file format: .pgdump) on their [API page](https://www.derpibooru.org/pages/data_dumps).

In [2]:
# Initializes the target folder where the data dump will be stored on my computer.

target_folder = "D:/repositories/MLpy/data_dump/"

In [3]:
# Calls the <download_most_recent_pgdump> function. 
# It will attempt to download the most recent dump available on the derpibooru website.

download_most_recent_pgdump(target_folder)

attempt 1: requesting pgdump for date 2020_10_30
the pgdump derpibooru_public_dump_2020_10_30.pgdump is already available locally.


### 2. Creating an empty DB for the source data

In [4]:
# Lists the commands to be passed to the postgreSQL instance

command_list = {"KILL all connections \ DROP DB derpibooru": "DROP DATABASE derpibooru WITH (FORCE);",
                "DROP DB derpi_lite"                       : "DROP DATABASE derpi_lite;",
                "CREATE DB derpibooru"                     : "CREATE DATABASE derpibooru;"}

In [5]:
# Connects to the postgreSQL instance and executes the list of commands listed above

connect_execute_disconnect_from_psql(filename="database.ini",
                                     section="postgresql",
                                     command_list=command_list)

Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
KILL all connections \ DROP DB derpibooru
DROP DB derpibooru
Failed: database "derpibooru" does not exist

DROP DB derpi_lite
CREATE DB derpibooru
Database connection closed.


### 3. Restoring the source DB

In [6]:
!ls data_dump -a

.
..
databases
derpibooru_public_dump_2020_10_30.pgdump


In [7]:
# Restores the db from the .pgdump file. It might take a long time to run.

restore_db(filename="database.ini",
           section="restoring_derpibooru",
           source="derpibooru_public_dump_2020_10_30.pgdump",
           target_folder=target_folder)

Restoring derpibooru -- Please wait
Restoration of derpibooru in process
This might take up to an hour on an hdd, please be patient and use psql to check process


This process took c. 25 minutes on the following machine:
- i5 4460
- 16gb ddr3
- GTX980ti
- 640gb WD HDD 7.2k rpm

In [ ]:
# Waits minutes
time.sleep(30*60)

### 4. Isolation inside new tables of the information we want to keep

In [8]:
# Lists the commands to be passed to the postgreSQL instance

command_list = {
    "CREATE TABLE new_images"        : "SELECT id, created_at, score, image_format, version_path INTO new_images " \
                                       "FROM images WHERE image_format IN ('png', 'PNG', 'jpg', 'JPEG', 'jpeg');", 
    "CREATE TABLE new_image_taggings": "SELECT image_id, tag_id INTO new_image_taggings FROM image_taggings " \
                                       "WHERE image_id IN (SELECT id FROM new_images);",
    "CREATE TABLE new_tags"          : "SELECT id, name INTO new_tags FROM tags WHERE id IN (SELECT tag_id FROM " \
                                       "new_image_taggings);"
}

In [9]:
# Connects to the postgreSQL instance and executes the list of commands listed above

connect_execute_disconnect_from_psql(filename="database.ini",
                                     section="derpibooru_full",
                                     command_list=command_list)

Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
CREATE TABLE new_images
CREATE TABLE new_image_taggings
CREATE TABLE new_tags
Database connection closed.


### 5. Removal of unneeded tables

In [ ]:
# Lists the tables to be deleted from the DB

tables = ["tags","images", "badge_awards","badges","comments","duplicate_reports","forums",
          "galleries","gallery_interactions","image_duplicates","image_faves","image_features",
          "image_hides","image_intensities","image_sources","image_taggings","posts",
          "source_changes","tag_aliases","tag_changes","tag_implications","topics","user_links",
          "users"]

In [10]:
# Connects to the postgreSQL instance and deletes each table listed in the variable above
# This takes a long time to run: ~25 minutes

for table in tables:
    connect_execute_disconnect_from_psql(filename="database.ini",
                                     section="derpibooru_full",
                                     command_list={f"DROP {table}": f"DROP TABLE {table} CASCADE"})

Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP tags
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP images
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP badge_awards
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP badges
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP comments
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP duplicate_reports
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP forums
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP galleries
Database connection closed.
Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
DROP gallery_interact

### 6. Renaming the DB

In [11]:
# Lists the commands to be passed to the postgreSQL instance

command_list = {"RENAME derpibooru": "ALTER DATABASE derpibooru RENAME TO derpi_lite;"}

In [12]:
# Connects to the postgreSQL instance and executes the list of commands listed above

connect_execute_disconnect_from_psql(filename="database.ini",
                                     section="postgresql",
                                     command_list=command_list)

Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
RENAME derpibooru
Database connection closed.


### 7. Saving the DB in a new .pgdump

In [13]:
# Saves the db into a .pgdump file. It might take a long time to run.

save_db(filename="database.ini", 
        section="postgresql", 
        filename_of_db="derpi_lite", 
        target_folder=target_folder)

Saving derpi_lite -- Please wait
Saving in process
This might take, please be patient and use psql to check process


This process took c. 5 minutes on the following machine:

In [16]:
# Waits minutes
time.sleep(10*60)

### 8. (OPTIONAL) Dropping the active DB

In [17]:
# Lists the tables to be deleted from the DB

command_list = {"KILL all connections \ DROP DB derpibooru": "DROP DATABASE derpi_lite WITH (FORCE);"}

In [18]:
# Connects to the postgreSQL instance and deletes each table listed in the variable above

connect_execute_disconnect_from_psql(filename="database.ini",
                                     section="postgresql",
                                     command_list=command_list)

Connecting to the PostgreSQL database...
ISOLATION_LEVEL_AUTOCOMMIT: 0
KILL all connections \ DROP DB derpibooru
Database connection closed.
